In [1]:
import numpy as np
from core.audio_io import load_audio, play_audio
from core.mixing import mix_sources_multichannel, error_signal
from core.visualization import plot_spectrogram
from separation.fastica_sep import fastica_separate

s1, fs = load_audio("data/speech/speaker1.wav")
s2, _  = load_audio("data/speech/speaker2.wav")

# Mix
A = np.array([[1.0, 0.3],
              [0.3, 1.0]])
X = mix_sources_multichannel([s1, s2], A)
x1, x2 = X[0], X[1]

# ICA
S_hat = fastica_separate(X, n_sources=2)
s1_hat, s2_hat = S_hat[0], S_hat[1]

# Listen
play_audio(x1, fs)



In [2]:
play_audio(s1_hat, fs)

In [3]:
play_audio(s1, fs)

In [4]:
play_audio(s2_hat, fs)

In [ ]:
from evaluation.benchmarking import evaluate_method, print_results
from core.audio_io import normalize

s1, fs = load_audio("data/speech/speaker1.wav")
s2, _  = load_audio("data/speech/speaker2.wav") 
s1 = normalize(s1)
s2 = normalize(s2)
# Mix
A = np.array([[1.0, 0.91],
              [0.83, 1.0]])

X = mix_sources_multichannel([s1, s2], A)

X_noisy = X + 0.01 * np.random.randn(*X.shape)
x1, x2 = X_noisy[0], X_noisy[1]

# ICA
S_hat = fastica_separate(X_noisy, n_sources=2)
s1_hat, s2_hat = S_hat[0], S_hat[1]

s1 = normalize(s1)
s2_hat = normalize(s2_hat)

# Listen
play_audio(x2, fs)

In [43]:
corr = np.corrcoef(s1, s1_hat)[0,1]
print(corr)

0.8517823001071141


In [46]:
play_audio(s1_hat, fs)

In [45]:
res_ica = evaluate_method(s1, x1, s1_hat)
print_results(res_ica)

SNR_in (dB)           : 2.862
SNR_out (dB)          : -21.956
SNR_improvement (dB)  : -24.817
Segmental_SNR (dB)    : -28.898
MSE                   : 0.878
LSD                   : 47.173


# Real-Life Data ICA Test 

In [8]:
import numpy as np
from enhancement.wiener_static import wiener_filter_static
from scipy.signal import correlate
from separation.fd_ica import fd_ica_separate


s1, fs = load_audio("data/real_recordings/ica_s1_sync.wav")
s2, _ = load_audio("data/real_recordings/ica_s2_sync.wav")

corr = correlate(s1, s2, mode="full")
lag = np.argmax(corr) - (len(s1) - 1)
print("Estimated delay (samples):", lag)
print("Delay (ms):", lag / fs * 1000)

if lag > 0:
    s1 = s1[lag:]
    s2 = s2[:len(s1)]
elif lag < 0:
    s2 = s2[-lag:]
    s1 = s1[:len(s2)]


Estimated delay (samples): -87
Delay (ms): -5.4375


In [9]:
X_real = np.vstack([s1, s2])  
# ICA
Sreal_hat = fastica_separate(X_real, n_sources=2)
sr1_hat, sr2_hat = Sreal_hat[0], Sreal_hat[1]


# Listen
play_audio(s1, fs)

In [10]:
play_audio(sr1_hat, fs)

In [11]:
corr = correlate(s1, s2, mode="full")
lag = np.argmax(corr) - (len(s1) - 1)
print("Estimated delay (samples):", lag)
print("Delay (ms):", lag / fs * 1000)

Estimated delay (samples): 0
Delay (ms): 0.0


In [12]:
lag

np.int64(0)